# Дообучение EasyOCR

In [27]:
import os
import fnmatch
import torch.backends.cudnn as cudnn
import torch
import yaml
from EasyOCR.trainer.train import train
from EasyOCR.trainer.utils import AttrDict
import pandas as pd

cudnn.benchmark = True
cudnn.deterministic = False

In [83]:
filename_dict = {'filename':[],'words':[]}
for file in os.listdir('/content/EasyOCR/trainer/all_data/train/img'):
    filename_dict['filename'].append(file)
    filename_dict['words'].append(file[:-4])
pd.DataFrame(filename_dict).to_csv('/content/EasyOCR/trainer/all_data/train/img/labels.csv', index = False)

filename_dict = {'filename':[],'words':[]}
for file in os.listdir('/content/EasyOCR/trainer/all_data/val/img'):
    filename_dict['filename'].append(file)
    filename_dict['words'].append(file[:-4]) 
pd.DataFrame(filename_dict).to_csv('/content/EasyOCR/trainer/all_data/val/img/labels.csv', index = False)  

In [84]:
def get_config(file_path):
    with open(file_path, 'r', encoding="utf8") as stream:
        opt = yaml.safe_load(stream)
    opt = AttrDict(opt)
    if opt.lang_char == 'None':
        characters = ''
        for data in opt['select_data'].split('-'):
            csv_path = os.path.join(opt['train_data'], data, 'labels.csv')
            df = pd.read_csv(csv_path, sep='^([^,]+),', engine='python', usecols=['filename', 'words'], keep_default_na=False)
            all_char = ''.join(df['words'])
            characters += ''.join(set(all_char))
        characters = sorted(set(characters))
        opt.character= ''.join(characters)
    else:
        opt.character = opt.number + opt.symbol + opt.lang_char
    os.makedirs(f'./saved_models/{opt.experiment_name}', exist_ok=True)
    return opt

In [87]:
opt = get_config("/content/custom_data_train.yaml")

In [88]:
train(opt, amp=False)

Filtering the images containing characters which are not in opt.character
Filtering the images whose label is longer than opt.batch_max_length
--------------------------------------------------------------------------------
dataset_root: /content/EasyOCR/trainer/all_data/train/img
opt.select_data: ['train']
opt.batch_ratio: ['1']
--------------------------------------------------------------------------------
dataset_root:    /content/EasyOCR/trainer/all_data/train/img	 dataset: train
/content/EasyOCR/trainer/all_data/train/img/
sub-directory:	/.	 num samples: 49382
num total samples of train: 49382 x 1.0 (total_data_usage_ratio) = 49382
num samples of train per batch: 64 x 1.0 (batch_ratio) = 64
--------------------------------------------------------------------------------
Total_batch_size: 64 = 64
--------------------------------------------------------------------------------
dataset_root:    /content/EasyOCR/trainer/all_data/val/img	 dataset: /
/content/EasyOCR/trainer/all_data/v

KeyboardInterrupt: ignored